In [1]:
from autonomos.utils.cache import Cache
from autonomos.utils.db import get_ctrs
from autonomos.dart.rank import evaluate, tracincp_valuation
from autonomos.dart.utils import split_dataset_by_qids
from allrank.config import Config
from autonomos.dart.types import SplitDataset
from autonomos.utils.attack import poison_ctrs, flip_label
import torch
import numpy as np
import random
from autonomos.datasets.aol import load_dataset

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
if hasattr(torch, 'mps'):
    torch.mps.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

config = Config.from_json("./allRank_config.json")
cache = Cache()

# df = load_dataset()
# cache.set("user_ids", df['user_id'].unique())

user_ids = cache.get("user_ids")
user_id = user_ids[0]
ctrs = get_ctrs(user_id)
print("Size of ctrs", len(ctrs))
ds = split_dataset_by_qids(ctrs, train_ratio=1/3, val_ratio=1/3)

evaluate(config, ds)

Size of ctrs 120


0.1464285714285714

In [7]:
from autonomos.dart.rank import shapley_valuation

shapley_valuation(config, ds, {
    'good_user_1': get_ctrs(user_ids[1]),
    'good_user_2': get_ctrs(user_ids[2]),
    'good_user_3': get_ctrs(user_ids[3]),
    'good_user_4': get_ctrs(user_ids[4]),
    'good_user_5': get_ctrs(user_ids[10]),
    'evil_user_1': poison_ctrs(flip_label, get_ctrs(user_ids[5])),
    'evil_user_2': poison_ctrs(flip_label, get_ctrs(user_ids[6])),
    'evil_user_3': poison_ctrs(flip_label, get_ctrs(user_ids[7])),
    'evil_user_4': poison_ctrs(flip_label, get_ctrs(user_ids[8])),
    'evil_user_5': poison_ctrs(flip_label, get_ctrs(user_ids[9])),
}, 100)

{'good_user_1': 6.256150793650794,
 'good_user_2': 2.366170634920635,
 'good_user_3': 27.91061507936508,
 'good_user_4': 8.565079365079361,
 'good_user_5': 21.866567460317466,
 'evil_user_1': -4.5683531746031765,
 'evil_user_2': 0.3626984126984119,
 'evil_user_3': -6.341369047619047,
 'evil_user_4': 0.2729166666666671,
 'evil_user_5': 4.775793650793651}

In [2]:
evil_ctrs = poison_ctrs(flip_label, get_ctrs(user_ids[1]))

mixed_ds = SplitDataset(
    train=ds.train + evil_ctrs + get_ctrs(user_ids[2]),
    vali=ds.vali,
    test=ds.test
)
evaluate(config, mixed_ds)

0.18725749559082894

In [3]:
import torch
from torch.utils.data import DataLoader
from autonomos.dart.rank import loss_batch_gradients
from autonomos.dart.types import SplitDataset, ClickThroughRecord
from allrank.config import Config
from autonomos.dart.rank import Model
import shutil
import uuid
import os
from copy import deepcopy
from autonomos.dart.utils import write_records, normalize_features
from allrank.models.model_utils import get_torch_device
from tqdm import tqdm
from typing import List, Dict, Type
from torch import Tensor

user_ds = ds
candidate_datasets = {
    'good1': get_ctrs(user_ids[2]),
    'good2': get_ctrs(user_ids[3]),
    'evil': evil_ctrs,
    'good3': get_ctrs(user_ids[4])
}

config = deepcopy(config)
dataset_path = f'.tmp/{uuid.uuid4().hex}'

mapped_train_ctrs = [
    (user_id, ctr) for user_id in candidate_datasets for ctr in candidate_datasets[user_id]
]
mapped_train_ctrs.extend([
    ('0', ctr) for ctr in user_ds.train
])

ds_split = SplitDataset(
    train=[ctr for _, ctr in mapped_train_ctrs],
    vali=user_ds.vali,
    test=user_ds.test
)
ds_split.shuffle()
write_records(dataset_path, ds_split)
normalize_features(dataset_path)
config.data.path = os.path.join(dataset_path, '_normalized')

dart = Model(config)
shutil.rmtree(dataset_path)
result = dart.train(trace=True)
checkpoints = result['checkpoints']

In [4]:
from autonomos.dart.tracin import vectorized_calculate_tracin_score

score_matrix = vectorized_calculate_tracin_score(
    dart.model,
    dart.loss_func,
    list(map(lambda x: x['state_dict'], checkpoints)),
    dart.train_dl,
    dart.test_dl,
    0.0001,
    config.data.slate_length,
    get_torch_device(),
    use_nested_loop_for_dot_product=False,
    float_labels=True
)

score_matrix

--> Precompute test gradients using checkpoints: 33it [00:26,  1.25it/s]


--> Calculate train influences (per doc)...:   5%|▌         | 1/19 [00:23<07:10, 23.90s/it]

--> Calculate train influences (per doc)...:  11%|█         | 2/19 [00:48<06:49, 24.10s/it]

--> Calculate train influences (per doc)...:  16%|█▌        | 3/19 [01:12<06:28, 24.30s/it]

--> Calculate train influences (per doc)...:  21%|██        | 4/19 [01:36<06:00, 24.06s/it]

...

--> Calculate train influences (per doc)...:  84%|████████▍ | 16/19 [06:23<01:11, 23.79s/it]

--> Calculate train influences (per doc)...:  89%|████████▉ | 17/19 [06:47<00:47, 23.85s/it]

--> Calculate train influences (per doc)...:  95%|█████████▍| 18/19 [07:11<00:23, 23.83s/it]

--> Calculate train influences (per doc)...: 100%|██████████| 19/19 [08:02<00:00, 25.41s/it]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
from autonomos.dart._tracin import tracincp

tracincp(config, ds, {
    'good1': get_ctrs(user_ids[2]),
    'good2': get_ctrs(user_ids[3]),
    'evil': evil_ctrs,
    'good3': get_ctrs(user_ids[4])
})

--> Precompute test gradients using checkpoints: 19it [00:17,  1.09it/s]
--> Precompute test gradients using checkpoints: 100%|██████████| 19/19 [00:00<00:00, 232.44it/s]


train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads

--> Calculate train...:   5%|▌         | 1/19 [00:15<04:31, 15.07s/it]

train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads

--> Precompute test gradients using checkpoints: 100%|██████████| 19/19 [00:00<00:00, 447.51it/s]


...

--> Calculate train...:  11%|█         | 2/19 [00:30<04:19, 15.28s/it]

train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads

--> Calculate train...:  11%|█         | 2/19 [00:45<06:28, 22.87s/it]

train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads shape: torch.Size([16, 44121])
train_grads shape: torch.Size([144, 44121])
test_grads

ValueError: operands could not be broadcast together with shapes (8,16) (144,16) 